<a href="https://colab.research.google.com/github/soumenkm/ML-Algorithms/blob/main/Transformers/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U "tensorflow-text==2.11.*"
%pip install -q tensorflow_datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 23.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.19.2 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tensorflow-datasets 4.9.4 requires protobuf>=3.20, but you have

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

import pdb
%pdb off

Automatic pdb calling has been turned OFF


In [3]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/51785 [00:00<?, ? examples/s]

...

Generating validation examples...:   0%|          | 0/1193 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteMDWML1/ted_hrlr_translate-vali…

Generating test examples...:   0%|          | 0/1803 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteMDWML1/ted_hrlr_translate-test…

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [4]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

184801/184801 [==============================] - 1s 3us/step


'./ted_hrlr_translate_pt_en_converter.zip'

In [5]:
tokenizers = tf.saved_model.load(model_name)
print([i for i in tokenizers.en.__dir__() if not i.startswith("_")]) # tokenizers methods
print(tokenizers.en.vocab[200:220]) # some tokens in the vocabulary
print(tokenizers.en.get_vocab_size()) # number of tokens in english vocabulary
print(tokenizers.pt.get_vocab_size()) # number of tokens in portugese vocabulary

['tokenizer', 'vocab', 'detokenize', 'get_reserved_tokens', 'get_vocab_path', 'get_vocab_size', 'lookup', 'tokenize']
tf.Tensor(
[b'different' b'around' b'thank' b'say' b'day' b'good' b'her' b'through'
 b'today' b'same' b'down' b'come' b'use' b'year' b'percent' b'kind' b'll'
 b'ca' b'three' b'called'], shape=(20,), dtype=string)
tf.Tensor(7010, shape=(), dtype=int32)
tf.Tensor(7765, shape=(), dtype=int32)


In [6]:
pt_examples, en_examples = train_examples.batch(3).__iter__().__next__()

print('1st batch byte string Examples in English:')
for i,en in enumerate(en_examples.numpy()):
    print(f"Example {i} --> ", en)

# print(en_examples.numpy()[0]) # bytes string
# print(en_examples.numpy()[0].decode()) # decoded string
# print(en_examples.numpy()[0].decode().encode()) # again encoded byte string

print()
print('1st batch string Examples in English:')
for i,en in enumerate(en_examples.numpy()):
    print(f"Example {i} --> ", en.decode('utf-8'))


1st batch byte string Examples in English:
Example 0 -->  b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
Example 1 -->  b'but what if it were active ?'
Example 2 -->  b"but they did n't test for curiosity ."

1st batch string Examples in English:
Example 0 -->  and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
Example 1 -->  but what if it were active ?
Example 2 -->  but they did n't test for curiosity .


In [7]:
encoded = tokenizers.en.tokenize(en_examples)

print('1st batch tokens ids Examples in English:')
for row in encoded.numpy():
    print(list(row))

print('1st batch token string Examples in English:')
for row in tokenizers.en.lookup(encoded):
    print(list(row.numpy()))

1st batch tokens ids Examples in English:
[2, 72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74, 2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15, 3]
[2, 87, 90, 107, 76, 129, 1852, 30, 3]
[2, 87, 83, 149, 50, 9, 56, 664, 85, 2512, 15, 3]
1st batch token string Examples in English:
[b'[START]', b'and', b'when', b'you', b'improve', b'search', b'##ability', b',', b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage', b'of', b'print', b',', b'which', b'is', b's', b'##ere', b'##nd', b'##ip', b'##ity', b'.', b'[END]']
[b'[START]', b'but', b'what', b'if', b'it', b'were', b'active', b'?', b'[END]']
[b'[START]', b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for', b'curiosity', b'.', b'[END]']


In [8]:
round_trip = tokenizers.en.detokenize(encoded)

print('1st batch detokenised binary string Examples in English:')
for line in round_trip.numpy():
    print(line)

1st batch detokenised binary string Examples in English:
b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
b'but what if it were active ?'
b"but they did n ' t test for curiosity ."


In [9]:
MAX_TOKENS=128
def prepare_batch(pt, en):
    """
    pt, en = train_examples.batch(BATCH_SIZE).__iter__().__next__()
    It takes one batch of BATCH_SIZE examples so pt is a list of BATCH_SIZE examples of portugese and same for en of english
    It returns the (x,y) as (pt, en_inputs) and en_labels where en_labels is the target output from decoder
    """

    pt = tokenizers.pt.tokenize(pt) # batch of tokenized examples (len = BATCH_SIZE)
    pt = pt[:,:MAX_TOKENS] # trim each example to MAX_TOKENS length
    pt = pt.to_tensor() # pad the other positions by 0 by converting rigged tensor to tensor

    en = tokenizers.en.tokenize(en) # batch of tokenized examples (len = BATCH_SIZE)
    en = en[:, :(MAX_TOKENS+1)] # trim each example to MAX_TOKENS length but take one extra position for shifting right
    en_inputs = en[:, :-1].to_tensor() # drop the end token (id 3) from input sequences but it will still be of max length MAX_TOKENS
    en_labels = en[:, 1:].to_tensor() # pad the other positions by 0 by converting ragged tensor to tensor and drop the start token (id 2)

    # print([en[i].numpy().shape[0] for i in range(3)]) # raw sequence will be of different length but max of MAX_TOKENS
    # print([en_inputs[i].numpy().shape[0] for i in range(3)]) # decoder inputs will be of same length (max of MAX_TOKENS) with only <start> token
    # print([en_labels[i].numpy().shape[0] for i in range(3)]) # target labels will be of same length (max of MAX_TOKENS) with no <start> token
    # print([pt[i].numpy().shape[0] for i in range(3)]) # encoder input sequence will be of same length but max of MAX_TOKENS

    return (pt, en_inputs), en_labels

In [10]:
BUFFER_SIZE = 1024
BATCH_SIZE = 64

def make_batches(ds):
    """
    ds = train_examples (of sizes 50000+)
    first it will shuffle the dataset randomly then it will make batches of examples and apply transformation
    to make proper (encoder inputs, decoder inputs) and decoder target output
    """

    ds = ds.shuffle(BUFFER_SIZE)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.map(prepare_batch)

    return ds

In [11]:
# Create training and validation set batches.
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

(x_pt, x_en), y_en = train_batches.__iter__().__next__()

In [12]:
def positional_encoding(sequence_length, embedding_depth):
    """
    The code below implements it, but instead of interleaving the sines and cosines,
    the vectors of sines and cosines are simply concatenated. Permuting the channels
    like this is functionally equivalent, and just a little easier to implement.
    """

    embedding_depth = embedding_depth/2
    positions = np.arange(sequence_length)[:, np.newaxis] # equivalent to reshaping to (sequence_length, 1)
    depths = np.arange(embedding_depth)[np.newaxis, :]/embedding_depth # equivalent to reshaping to (1, embedding_depth)

    angle_rates = 1 / (10000**depths) # (1, embedding_depth)
    angle_rads = positions * angle_rates # (sequence_length, embedding_depth) [both the vectors are broadcasted to become a matrix]

    pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)

    return tf.cast(pos_encoding, dtype=tf.float32)

In [13]:
class Embedding(tf.keras.layers.Layer):

    def __init__(self, vocab_size, d_model):
        """
        It does both word embedding and positional embedding and then add the two
        embedding size = d_model (typically 512)
        vocabulary size is needed for word embeddings
        """

        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size

        self.word_embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size,
                                                   output_dim=self.d_model,
                                                   mask_zero=True) # mask_zero is true means it will treat 0 as padding

        self.pos_encoding = positional_encoding(sequence_length=2048, embedding_depth=self.d_model) # it can handle max 2048 sequence

    def compute_mask(self, *args):
        """
        layer.embedding.compute_mask(tf.constant([[1,2,3],[0,0,8],[9,9,0]]))
        returns elements in the input sequences should be considered valid (True) and which should be masked (False)
        """

        return self.word_embedding.compute_mask(*args)

    def call(self, x):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_pt is of shape (BATCH_SIZE, N) where N is variable sequence length
        here x will be x_pt or x_en
        this x will be passed to word embedding layer and positional embedding will be added
        returns embedding of the batch (BATCH_SIZE, N, d_model)
        """

        N = tf.shape(x)[1]

        x = self.word_embedding(x) # shape of output x will be increased by 1 dimension (BATCH_SIZE, N) --> (BATCH_SIZE, N, d_model)
        x = x * tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # this factor sets the relative scale of the word embedding and position encoding
        x = x + self.pos_encoding[tf.newaxis, :N, :self.d_model] # same as reshaping to (BATCH_SIZE, N, d_model)

        return x

embed_pt = Embedding(vocab_size=tokenizers.pt.get_vocab_size(), d_model=512)
embed_en = Embedding(vocab_size=tokenizers.en.get_vocab_size(), d_model=512)

pt_emb = embed_pt(x_pt)
en_emb = embed_en(x_en)

In [14]:
class BaseAttention(tf.keras.layers.Layer):

    def __init__(self, num_heads, d_model):

        super().__init__()

        self.num_heads = num_heads
        self.d_model = d_model
        self.key_dim = int(self.d_model / self.num_heads)
        self.value_dim = self.key_dim

        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim, value_dim=self.value_dim)
        self.add = tf.keras.layers.Add()
        self.layernorm = tf.keras.layers.LayerNormalization()

    def call(self, query, key, value, is_causal):

        attn_output, attn_scores = self.mha(query=query,
                                            key=key,
                                            value=value,
                                            return_attention_scores=True,
                                            use_causal_mask=is_causal) # call arguments of MultiHeadAttention object

        # Cache the attention scores for plotting later.
        self.last_attn_scores = attn_scores

        x = self.add([query, attn_output]) # call arguments of Add object
        x = self.layernorm(x) # call arguments of LayerNormalization object

        return x

In [15]:
class CrossAttention(BaseAttention):

    def __init__(self, num_heads, d_model):

        super().__init__(num_heads, d_model)

    def call(self, x, context):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_pt is of shape (BATCH_SIZE, N) where N is variable sequence length
        call "call" method on PositionEmbedding object that returns embedding of the batch (BATCH_SIZE, N, d_model) -> x.shape and context.shape
        x will be the decoder side embedding (query), context will be the encoder side output (key and value)
        it will return the cross attended output after add and norm applied
        """

        x = super().call(x, context, context, False)

        return x

In [16]:
class GlobalSelfAttention(BaseAttention):

    def __init__(self, num_heads, d_model):

        super().__init__(num_heads, d_model)

    def call(self, x):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_pt is of shape (BATCH_SIZE, N) where N is variable sequence length
        call "call" method on PositionEmbedding object that returns embedding of the batch (BATCH_SIZE, N, d_model) -> x.shape
        x will be the encoder side embedding (query, key, value all will be same)
        it will return the global self attended output after add and norm applied
        """

        x = super().call(x, x, x, False)

        return x

In [17]:
class CausalSelfAttention(BaseAttention):

    def __init__(self, num_heads, d_model):

        super().__init__(num_heads, d_model)

    def call(self, x):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_en is of shape (BATCH_SIZE, N) where N is variable sequence length
        call "call" method on PositionEmbedding object that returns embedding of the batch (BATCH_SIZE, N, d_model) -> x.shape
        x will be the decoder side embedding (query, key, value all will be same)
        it will return the causal self attended output after add and norm applied
        """

        x = super().call(x, x, x, True)

        return x

In [18]:
class FeedForward(tf.keras.layers.Layer):

    def __init__(self, d_model, dff):
        """
        d_model is the number of units in input layer and dff is number of neurons in 1st hidden layer
        d_model is again the number of neurons in the output layer with linear activations
        """

        super().__init__()

        self.d_model = d_model
        self.dff = dff

        self.seq = tf.keras.Sequential([
            tf.keras.layers.Dense(dff, activation='relu'),
            tf.keras.layers.Dense(d_model, activation='linear')
        ])

        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_en is of shape (BATCH_SIZE, N) where N is variable sequence length
        call "call" method on PositionEmbedding object that returns embedding of the batch (BATCH_SIZE, N, d_model) -> x.shape
        x will be the encoder side self attended output or decoder side cross attended output
        it will return the encoder or decoder output after add and norm applied
        """

        a = self.seq(x)
        x = self.add([x, a])
        x = self.layer_norm(x)

        return x

In [19]:
class EncoderLayer(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, dff):
        """
        encoder hyper parameters are d_model (embedding size),
        num_heads and dff for hidden layer neuron units
        """

        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff

        self.self_attention = GlobalSelfAttention(num_heads=self.num_heads,
                                                  d_model=self.d_model)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_pt is of shape (BATCH_SIZE, N) where N is variable sequence length
        call "call" method on PositionEmbedding object that returns embedding of the batch (BATCH_SIZE, N, d_model) -> x.shape
        x will be the encoder side embedding output
        it will return the encoder side output
        """

        x = self.self_attention(x)
        x = self.ffn(x)

        return x

In [121]:
NUM_LAYERS = 1 # 6
D_MODEL = 8 # 512
NUM_HEADS = 1 # 8
DFF = 16 # 2048
VOCAB_SIZE = 10000

class Encoder(tf.keras.layers.Layer):

    def __init__(self, num_layers=NUM_LAYERS, d_model=D_MODEL, num_heads=NUM_HEADS, dff=DFF, vocab_size=VOCAB_SIZE):

        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.vocab_size = vocab_size

        self.pos_embedding = Embedding(vocab_size=self.vocab_size, d_model=self.d_model)

        self.enc_layers = [] # stack all the encoder layers on top of another
        for i in range(self.num_layers):
            self.enc_layers.append(EncoderLayer(d_model=self.d_model, num_heads=self.num_heads, dff=self.dff))

    def call(self, x):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_pt is of shape (BATCH_SIZE, N) where N is variable sequence length
        here x will be x_pt
        returns encoding of the batch (BATCH_SIZE, N, d_model)
        """

        x = self.pos_embedding(x)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x)

        return x

In [122]:
class DecoderLayer(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, dff):
        """
        decoder hyper parameters are d_model (embedding size),
        num_heads and dff for hidden layer neuron units
        """

        super().__init__()

        self.d_model = d_model
        self.num_heads = num_heads
        self.dff = dff

        self.causal_self_attention = CausalSelfAttention(num_heads=self.num_heads,
                                                         d_model=self.d_model)

        self.cross_attention = CrossAttention(num_heads=self.num_heads,
                                              d_model=d_model)

        self.ffn = FeedForward(d_model, dff)

    def call(self, x, context):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_en is of shape (BATCH_SIZE, N) where N is variable sequence length
        call "call" method on PositionEmbedding object that returns embedding of the batch (BATCH_SIZE, N, d_model) -> x.shape
        x will be the encoder side embedding output, context will be the encode side output with same shape as x
        it will return the decoder side output
        """

        x = self.causal_self_attention(x)
        x = self.cross_attention(x, context)
        x = self.ffn(x)

        return x

In [123]:
class Decoder(tf.keras.layers.Layer):

    def __init__(self, num_layers=NUM_LAYERS, d_model=D_MODEL, num_heads=NUM_HEADS, dff=DFF, vocab_size=VOCAB_SIZE):

        super().__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.vocab_size = vocab_size

        self.pos_embedding = Embedding(vocab_size=self.vocab_size, d_model=self.d_model)

        self.dec_layers = [] # stack all the decoder layers on top of another
        for i in range(self.num_layers):
            self.dec_layers.append(DecoderLayer(d_model=self.d_model, num_heads=self.num_heads, dff=self.dff))

    def call(self, x, context):
        """
        (x_pt, x_en), y_en = train_batches.__iter__().__next__()
        x_en is of shape (BATCH_SIZE, N) where N is variable sequence length
        here x will be x_en and context will be encoder side output
        returns encoding of the batch (BATCH_SIZE, N, d_model)
        """

        x = self.pos_embedding(x)

        for i in range(self.num_layers):
            x = self.dec_layers[i](x, context)

        return x

In [124]:
class Transformer(tf.keras.Model):

    def __init__(self, target_vocab_size):

        super().__init__()

        self.encoder = Encoder()
        self.decoder = Decoder()
        self.final_layer = tf.keras.layers.Dense(target_vocab_size, activation='linear') # prob are small

    def call(self, inputs):
        # To use a Keras model with `.fit` you must pass all your inputs in the
        # first argument.

        context, x  = inputs

        context = self.encoder(context)  # (batch_size, context_len, d_model)
        x = self.decoder(x, context)  # (batch_size, target_len, d_model)

        # Final linear layer output.
        logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

        try:
        # Drop the keras mask, so it doesn't scale the losses/metrics.
        # b/250038731 (bug)
            del logits._keras_mask
        except AttributeError:
            pass

        # Return the final output and the attention weights.
        return logits

transformer = Transformer(target_vocab_size=tokenizers.en.get_vocab_size().numpy())
output = transformer((x_pt, x_en))

print(x_en.shape)
print(x_pt.shape)
print(output.shape)

(64, 90)
(64, 96)
(64, 90, 7010)


In [125]:
transformer.summary()

Model: "transformer_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_15 (Encoder)        multiple                  80600     
                                                                 
 decoder_15 (Decoder)        multiple                  80904     
                                                                 
 dense_171 (Dense)           multiple                  63090     
                                                                 
Total params: 224,594
Trainable params: 224,594
Non-trainable params: 0
_________________________________________________________________


In [126]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

    def __init__(self, d_model=D_MODEL, warmup_steps=4000):
        """
        Learning rate changes with respect to training steps so we need a custom schedule
        lrate = d_model^(-0.5) * min(step_num^(-0.5), step_num * warmup_steps^(-1.5))
        till step of 4000, the lrate is linear and touches maximum of 0.0014
        then it decreases hyperbolically
        """

        super().__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):

        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule()
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [127]:
def masked_loss(label, pred):
    """
    Since the target sequences (label starting without <sos> token) are padded,
    it is important to apply a padding mask when calculating the loss.
    Use the sparse catergorical cross-entropy loss function
    label shape is (BATCH_SIZE, N)
    pred shape is (BATCH_SIZE, N, TARGET_VOCAB)
    """

    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                reduction='none') # for custom training loop, loss reduction should be none/sum
    loss = loss_object(label, pred) # call the loss object to calculate the loss

    mask = tf.cast(mask, dtype=loss.dtype) # convert mask to float
    loss = loss * mask # will be zero if mask is false (label == 0)

    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask) # by default the loss is averaged over all dimensions (batch and sequence)

    return loss

In [128]:
def masked_accuracy(label, pred):
    """
    label shape is (BATCH_SIZE, N)
    pred shape is (BATCH_SIZE, N, TARGET_VOCAB)
    """

    pred = tf.argmax(pred, axis=-1) # calculates the vocab index of maximum logits (pred token)
    label = tf.cast(label, pred.dtype) # contains vocab index (true target token)

    matching = label == pred # it will be a boolean tensor where 1 means label == pred
    mask = label != 0 # mask = 1 means the value should be considered for calculating accuracy

    matching = matching & mask # element wise AND operation
    matching = tf.cast(matching, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)

    return tf.reduce_sum(matching)/tf.reduce_sum(mask) # average accuracy

In [129]:
transformer.compile(loss=masked_loss, optimizer=optimizer, metrics=[masked_accuracy])
history = transformer.fit(x=train_batches, epochs=3, verbose=1, validation_data=val_batches)

Epoch 1/3
473/810 [================>.............] - ETA: 4:27 - loss: 8.2911 - masked_accuracy: 0.0312

KeyboardInterrupt: ignored

In [120]:
class Translator:

    def __init__(self, tokenizers, transformer):

        self.tokenizers = tokenizers
        self.transformer = transformer

    def __call__(self, source, ground_truth=None):
        """
        source = 'este é um problema que temos que resolver.'
        ground_truth = 'this is a problem we have to solve .'
        """

        sentence = self.tokenizers.pt.tokenize(tf.constant(source)[tf.newaxis]).to_tensor() # without batch axis, it throws an error

        # As the output language is English, initialize the output with the English `[START]` token.
        start_decoder_input = self.tokenizers.en.tokenize(tf.constant("")[tf.newaxis]).to_tensor() # without batch axis, it throws an error
        start_token = start_decoder_input[:,0:1]
        end_token = start_decoder_input[:,:-1]
        decoder_input = start_token

        for i in range(MAX_TOKENS):

            output = self.transformer((sentence, decoder_input))
            decoder_output_token = tf.math.argmax(output, axis=-1)
            generated_token = decoder_output_token[:,-1:]
            decoder_input = tf.concat([decoder_input, generated_token], axis=-1)

            if (generated_token[:,0] == end_token[:,0]) or (i >= tf.shape(sentence)[1].numpy() * 2):
                break

        prediction = self.tokenizers.en.detokenize(decoder_input).numpy()[0].decode()

        return {"source": source,
                "prediction": prediction,
                "ground truth": ground_truth}

transformer = Transformer(target_vocab_size=tokenizers.en.get_vocab_size().numpy())
translator = Translator(tokenizers, transformer)
translator(source = 'este é um problema que temos que resolver.',
           ground_truth = 'this is a problem we have to solve .')

{'source': 'este é um problema que temos que resolver.',
 'prediction': 'heroic apply witnessed email email email oh coca coca cocaired coca visual visual visual visual visual visual visual visual visual email email',
 'ground truth': 'this is a problem we have to solve .'}